<a href="https://colab.research.google.com/github/tort-cam/ST-554-P1/blob/main/Task2/ST554Proj1EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Project 1 Task 2: Exploratory Data Analysis
Author : Cameron Mullaney

In this notebook, we will perform an **EDA** (exploratory data analysis) on the **Air Quality dataset** provided by UCI.

This process features a few key steps:
- Loading in the data
- Basic data validation
- Investigate missing values
- Clean up data (Handle missing values)
- Investigate distributions (the fun part!)
    - This is where we will create some preliminary figures, and try to get a general idea of the patterns that may exist in this data
    - Our exploration revolves around the **Benzene levels** specifically, which dramatically reduces the number of relationships we have to explore.
    - This section will heavily feature plots, as well as single and multiple linear regression models.


This process will provide us direction for future work with this dataset, better informing the variables we would choose to investigate further.


##Loading in the data from UCI and installing modules I'll need.

Because I am using a variety of methods to evaluate the data, I am using *quite a few* different modules.

In [13]:
!pip install ucimlrepo ## This module allows us to pull in the UCI data

In [15]:
## These will allow us to handle, visualize and analyze our data.
import numpy as np
import pandas as pd
import math
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from sklearn import linear_model
import sklearn.metrics as metrics
import plotly.express as px
from sklearn.model_selection import train_test_split

In [18]:
air_quality = fetch_ucirepo(id=360) #Pull in UCI data
aq = pd.DataFrame(air_quality.data.features)
#This air_quality object contains 2 separate datasets,
#and we only need one, the "features" subset.

ConnectionError: Error connecting to server

##Understanding how this data is stored
This data is almost entirely numeric, with the exception of `Date` and `Time`, and I will convert those to a usable value later. All other columns are either integers or floats.

In [ ]:
aq.info()

We were told we can ignore the "True" pollutant measurements, other than Benzene, so I'm removing columns with a "(GT)" in them, other than C6H6 (Benzene)

In [ ]:
aq_sub = aq.iloc[:,[0,1,3,5,6,8,10,11,12,13,14]] ## Keeping all rows, but only columns 0, 1, 3, etc.

Taking a look at aq_sub, it seems good! Removed what we don't need.

In [ ]:
aq_sub

##Basic Data Validation

Taking a quick peek at what we're working with, using the `.describe()` method.
Looks like there are a lot of -200 values, which represent "missing values"

In [ ]:
aq_sub.describe()

##Determine Rate of Missing Values

Let's see how many missing values there are: Looks like we've got ~3300, but only in 366 rows. This suggests that these missing values are often stacked in single rows. Here, `na_ct` will count missing values, while `row_ct` will count the number of rows containing missing values.

In [ ]:
na_ct = 0 ## Count of missing values
row_ct = 0 ## Number of rows with >0 missing values
test = False ## Boolean for each row's status
for row in aq_sub.iloc[:,2:].values: ## Ignore "Date" and "Time" Columns
  test = False ## Reset to False every loop
  for val in row:
    if math.isclose(val, -200): ## Add 1 to na_ct for every "-200"
      na_ct+=1
      test = True ## If we find a missing value, this marks the row
  if test:
    row_ct+=1 ## After each row, if test == True, that row contains a missing value, so +1 to row count.
print ("Missing Values:\t",str(na_ct))
print ("Rows w/ Missing Values:\t", row_ct)

##Clean up data

Okay! Let's replace these -200 values with `NaN` values, and then we can use `.dropna()` to remove them. \
Our df has gotten 366 rows shorter, the number we were expecting from the last cell's `row_ct` value. I've decided on `aqf` as our dataFrame name, for "Air Quality Fixed".
We've also renamed our columns for clarity. Now, another look at `.describe()` shows us meaningful summaries of our data: We know each variables mean, median, st.dev and much more!

In [ ]:
aqtemp = aq_sub.replace(float(-200), np.nan) ## Replace "-200" with "NaN"
aqf = aqtemp.dropna() ## Get rid of all rows with an "NaN"
print(len(aq_sub)-len(aqf))
aqf = aqf.rename(columns={'C6H6(GT)': 'B', 'PT08.S1(CO)': 'CO', \
                          'PT08.S2(NMHC)': 'NMHC', 'PT08.S3(NOx)': 'NOx', \
                          'PT08.S4(NO2)': 'NO2', 'PT08.S5(O3)': 'O3'})
aqf.describe()

Here I am reformatting date and time into a single column for simplicity, and using `datetime` to treat them as a datetime object.

In [ ]:
aqf["DTtemp"] = aqf['Date'] + " " + aqf["Time"] ##New column which concatenates "Date" and "Time"
aqf["datetime"] = pd.to_datetime((aqf['DTtemp']), format = 'mixed', dayfirst = False)
aqf = aqf.drop("DTtemp", axis = 1)

##Investigate Distributions

###Benzene Histogram
Now that we've cleaned up these missing values, let's get to work! Looks like Benzene (B) is pretty right skewed, with most values being <20.


In [ ]:
plt.hist(aqf["B"])

###Benzene by Datetime
Looks like the `datetime` column is working as intended! Noticing a drop in Benzene values around August 2004

In [ ]:
plt.scatter(aqf["datetime"], aqf["B"], s = .05, c = "black")
plt.title("Benzene levels over time")

###Benzene by Everything (W/ SLR)
Here we've got scatter plots of Benzene levels with every other numeric variable, just to get an idea of what our data looks like. Immediately, it seems like Benzene is correlated strongly with CO, NOx, NO2, and O3. NMHC looks a little *too* nice, where it seems like they might be measuring the same thing, just on different scales. I'll need some software math to figure out if temp, relative humidity, or absolute humidity are correlated with Benzene.

In [ ]:
notB = aqf.columns.drop(["B", "Date", "Time", "datetime"])
every = sns.pairplot(kind = "reg", data = aqf, y_vars = "B", x_vars = notB, markers = "+",\
                    plot_kws = {'line_kws':{'lw' : 2, 'color':"darkblue"}, 'scatter_kws':{'alpha':.1, 's':2}})

###Datetime by Everything
Here, I've set the Y axis to `datetime`, showing changes in these variables over the 1 year range. Seems like a lot of the pollution variables drop in August 2004.

In [ ]:
notDT = aqf.columns.drop(["Date", "Time", "datetime"]) ## List of columns to avoid in this set of plots
every = sns.pairplot(kind = "scatter", data = aqf, y_vars = "datetime", x_vars = notDT, markers = "+", plot_kws = {'s' : 2})

###Simple (Single) Linear Models
Let's look at some linear model values!
Here we see that NMHC and Benzene are very strongly correlated (Rsquared of .965), which makes sense given their *too* nice graph earlier.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(aqf['NMHC'], aqf['B'], test_size=0.2, random_state=42)
co = linear_model.LinearRegression()
co.fit(X_train.values.reshape(-1,1), y_train.values)
COpred = co.predict(X_train.values.reshape(-1,1))
print("Linear Model for NMHC and Benzene", "\nIntercept \t:", co.intercept_, "\nCoefficient\t:", co.coef_, \
      "\nRMSE\t\t:", np.sqrt(metrics.mean_squared_error(y_train.values, COpred)), "\nRsquared\t:", \
      co.score(X_test.values.reshape(-1,1), y_test))

Here I've put together a function we can use to find some simple linear regression values. Just need to provide the column names of the 2 variables.

In [ ]:
def lmtest(v1, v2, graph = False):
  """
  This function takes in two column names from this data
  (NOT ANY DATA SHEET JUST THIS ONE)
  and returns some important regression values, as well as a color-coded figure
  if you supply True for graph
  """
  X_train, X_test, y_train, y_test = train_test_split(aqf[v1], aqf[v2], test_size=0.2, random_state=42) ## Separate out data into train and test sets

  lm = linear_model.LinearRegression()
  lm.fit(X_train.values.reshape(-1,1), y_train.values)
  if graph: ## If it's wanted, this will produce a best fit line for the data.
    sns.regplot(x = X_train, y = y_train, marker = "+", scatter_kws = {'s' : 1}, \
              line_kws ={'color': "darkblue"}, label = "Train")
    sns.scatterplot(x = X_test, y = y_test, marker = "o", s = 1, color = "red", label = "Test")
    plt.legend()
  pred = lm.predict(X_train.values.reshape(-1,1))
  print("Linear Model for " + v1 + " and " + v2, \
        "\nIntercept \t:", lm.intercept_, \
        "\nCoefficient\t:", lm.coef_, \
        "\nRMSE\t\t:", np.sqrt(metrics.mean_squared_error(y_train.values, pred)), \
        "\nRsquared\t:", lm.score(X_test.values.reshape(-1,1), y_test))

Here, I am using this `lmtest` function to look at the relationship between NO2 and Benzene

In [ ]:
lmtest("NO2", "B")

###Creating Categoricals
Creating ordered categories for Temp using `.describe()` quartiles from earlier, separating out the top and bottom 25% of values as "low" and "high"

######Temperature

In [ ]:
aqf['Tcat'] = "M"
aqf.loc[aqf['T']> 24.4, 'Tcat'] = "H" ##Top 25% of temps considered high
aqf.loc[aqf['T']< 11.8, 'Tcat'] = "L" ## Bottom 25% considered low
aqf['Tcat'] = aqf.Tcat.astype("category")
order = ["L", "M", "H"]
cat_dtype = pd.CategoricalDtype(categories=order, ordered=True)
aqf['Tcat'] = aqf['Tcat'].astype(cat_dtype) ## We will treat this "Tcat" column as a category

Here, we see that in general, as temp increases (From low -> medium -> high), mean Benzene level also increases.

This is reflected in the numeric summary as well.

In [ ]:
aqf.boxplot(column = ['B'], by = "Tcat")
plt.title("Benzene by Temperature")
plt.xlabel("Temperature Level (High, Low, Medium)")
aqf.groupby("Tcat", observed = False)["B"].mean()

##### Relative Humidity

In [ ]:
aqf['RHcat'] = "M"
aqf.loc[aqf['RH']> 62.5, 'RHcat'] = "H" ##Same as for temperature
aqf.loc[aqf['RH']< 35.8, 'RHcat'] = "L"
aqf['RHcat'] = aqf.RHcat.astype("category")
aqf['RHcat'] = aqf['RHcat'].astype(cat_dtype)

We don't see nearly as much change as Relative Humidity increases, though there is still a slight change.

In [ ]:
aqf.boxplot(column = ['B'], by = "RHcat")
plt.title("Benzene by Relative Humidity")
plt.xlabel("Temperature Level (High, Low, Medium)")
aqf.groupby("RHcat", observed = False)["B"].mean()

#####Months and Seasons

In [ ]:
aqf['Month'] = pd.DatetimeIndex(aqf['datetime']).month
cat_dtype2 = pd.CategoricalDtype(ordered=True)
aqf['Month'] = aqf['Month'].astype(cat_dtype2)
##This column is numbers, but we are treating it as an ordered category
##Don't want to accidentally treat months as numbers (add, subtract, etc.)

In [ ]:
aqf["Season"] = "Fall" ## All months get "fall"
aqf.loc[aqf['Month']< 10,'Season'] = "Summer"## If month is before october, it gets "summer"
aqf.loc[aqf['Month']< 7, 'Season'] = "Spring"## We go down the list editing only the lower months
aqf.loc[aqf['Month']< 4, 'Season'] = "Winter"## until they're all categorized.
aqf['Season'] = aqf.Season.astype("category")
order2 = ["Winter", "Spring", "Summer", "Fall"]
cat_dtype = pd.CategoricalDtype(categories=order2, ordered=True)
aqf['Season'] = aqf['Season'].astype(cat_dtype)

We do see some seasonal and monthly variation in Benzene!

In [ ]:
aqf.boxplot(column = ['B'], by = "Season")

In [ ]:
aqf.boxplot(column = ['B'], by = "Month")

In [ ]:
sns.regplot(x = aqf['Month'], y = aqf['B'], order = 8, marker = "none", color = "red")


### Correlations

Here I've printed the Spearman's correlation values for Benzene with each other numeric variable (no dates or times). What we see here reflects the figures early on, where the other pollutants are much more clearly correlated with Benzene than temperature or humidity.

In [ ]:
notB = aqf.columns.drop(["B", "Date", "Time", "datetime", "Tcat", "Month", "Season"])
print("Spearman's Corr Values \n")
for x in notB:
  print(x, "\n", aqf.loc[:, ["B", x]].corr(method = 'spearman'), "\n")


###MLR
Here, I have put together a MLR of NOx, NO2, with Benzene. We get a RMSE of 3.84

In [ ]:
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(aqf['NOx'], aqf['NO2'], aqf['B'], test_size=0.2, random_state=42)

mlr = linear_model.LinearRegression()
x_train = pd.concat([X1_train, X2_train], axis = 1) ##With two "x" variables, I have to treat the data set a little differently
x_test = pd.concat([X1_test, X2_test], axis = 1)    ## Here I'm concatenating x1 and x2 together so I can feed them into the model.
mlr.fit(x_train, y_train)
pred = mlr.predict(x_train)

print("Benzene by",X1_train.name, "and", X2_train.name, \
      "\nIntercept \t\t:", mlr.intercept_, \
      "\nCoefficient [NOx, NO2]\t:", mlr.coef_, \
      "\nRMSE\t\t\t:", np.sqrt(metrics.mean_squared_error(y_train.values, pred)), \
      "\nRsquared\t\t:", mlr.score(x_test, y_test))

This is a 3D *interactive* figure of NOx, NO2, and Benzene, the same variables used in the MLR test above. These variables seem pretty related.

In [ ]:
fig = plt.figure()
fig = px.scatter_3d(aqf, x = 'NOx', y = 'NO2', z = 'B')
fig.update_traces(marker=dict(size=1))

Here, using Temp and Relative Humidity to predict B, we get a much larger RMSE of 7.25, suggesting that this model is not as helpful.

In [ ]:
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(aqf['T'], aqf['RH'], aqf['B'], test_size=0.2, random_state=42)

mlr = linear_model.LinearRegression()
x_train = pd.concat([X1_train, X2_train], axis = 1)
x_test = pd.concat([X1_test, X2_test], axis = 1)
mlr.fit(x_train, y_train)
pred = mlr.predict(x_train)

print("Benzene by",X1_train.name, "and", X2_train.name, \
      "\nIntercept \t\t:", mlr.intercept_, \
      "\nCoefficient [NOx, NO2]\t:", mlr.coef_, \
      "\nRMSE\t\t\t:", np.sqrt(metrics.mean_squared_error(y_train.values, pred)), \
      "\nRsquared\t\t:", mlr.score(x_test, y_test))